In [68]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries
using CoordinateTransformations
using GeometryBasics

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [79]:
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
vis = Visualizer()
render(vis)
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file_alpha = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmFixedJaws.urdf")
urdf_file_water = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmWater.urdf")
# mechanism_alpha = parse_urdf(urdf_file_alpha; gravity = [0.0, 0.0, 0.0])
mechanism_alpha = parse_urdf(urdf_file_alpha)
mechanism_water = parse_urdf(urdf_file_water; gravity = [0.0, 0.0, 0.0])

delete!(vis)

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file_alpha)
water_vis = URDFVisuals(urdf_file_water)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file_alpha), vis[:alpha])
mvis_water = MechanismVisualizer(mechanism_water, URDFVisuals(urdf_file_water))
# print(typeof(mvis_alpha))

# Name the joints of the mechanisms
base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
base_j_water, shoulder_j_water, elbow_j_water, wrist_j_water = joints(mechanism_water)
~, shoulder_body, upper_arm_body, elbow_body, wrist_body = bodies(mechanism_alpha)

shoulder_frame = default_frame(shoulder_body)
upper_arm_frame = default_frame(upper_arm_body)
elbow_frame = default_frame(elbow_body)
wrist_frame = default_frame(wrist_body)
base_frame = root_frame(mechanism_alpha)
# # last_link = bodies(mechanism_alpha)[end]
# # body_frame = default_frame(last_link)

frame_names = ["shoulder_cob", "ua_cob", "elbow_cob", "wrist_cob"]
vecs = [SVector{3, Float64}([-0.001, -0.003, .032]), SVector{3, Float64}([0.073, 0.0, -0.002]), SVector{3, Float64}([0.015, -0.012, -.003]), SVector{3, Float64}([0.0, 0.0, -.098])]
cob_frames = []
for i in 1:4
    bod = bodies(mechanism_alpha)[i+1]
    frame = CartesianFrame3D(frame_names[i])
    vec = vecs[i]
    cob_transform = Transform3D(frame, default_frame(bod), vec)
    if !(is_fixed_to_body(bod, frame))
        add_frame!(bod, cob_transform)
        push!(cob_frames, frame)
        # setelement!(mvis_alpha, frame)    # visualizes COB frames in MeshCat
    end
end

println("URDFs parsed. Visualizers ready. \n")

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8738
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


URDFs parsed. Visualizers ready. 



┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8739
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


In [80]:
# ------------------------------------------------------------------------
#                           BUOYANCY SETUP
# ------------------------------------------------------------------------
# f = rho * g * V
# f = 997 (kg/m^3) * 9.81 (m/s^2) * V_in_L *.001 (m^3) = kg m / s^2
# One time setup of buoyancy forces
volumes = [.018, .203, .025, .155] # shoulder, ua, elbow, wrist 
# volumes = [0.0, 0.0, 0.0, 0.0]
buoy_force_mags = volumes * 997 * 9.81 * .001
buoy_lin_forces = []
for mag in buoy_force_mags
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, mag])
    push!(buoy_lin_forces, lin_force)
end

# println(spatial_inertia(shoulder_body))
# println(spatial_inertia(upper_arm_body))
# println(spatial_inertia(elbow_body))
# println(spatial_inertia(wrist_body))
# println(mass(mechanism_alpha))

In [88]:
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
function impose_alpha_state_on_water(state_alpha, state_water)
    """
    Take the current state (configuration and velocity) of the alpha arm and impose it on the water mechanism state.
    """
    set_configuration!(state_water, base_j_water, configuration(state_alpha, base_joint))
    set_configuration!(state_water, shoulder_j_water, configuration(state_alpha, shoulder_joint))
    set_configuration!(state_water, elbow_j_water, configuration(state_alpha, elbow_joint))
    set_configuration!(state_water, wrist_j_water, configuration(state_alpha, wrist_joint))
    set_velocity!(state_water, base_j_water, velocity(state_alpha, base_joint))
    set_velocity!(state_water, shoulder_j_water, velocity(state_alpha, shoulder_joint))
    set_velocity!(state_water, elbow_j_water, velocity(state_alpha, elbow_joint))
    set_velocity!(state_water, wrist_j_water, velocity(state_alpha, wrist_joint))
end;

function impose_alpha_desacc_on_water(des_acc, des_acc_water)
    des_acc_water[base_j_water][1] = des_acc[base_joint][1]
    des_acc_water[shoulder_j_water][1] = des_acc[shoulder_joint][1]
    des_acc_water[elbow_j_water][1] = des_acc[elbow_joint][1]
    des_acc_water[wrist_j_water][1] = des_acc[wrist_joint][1]
end;

# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
# set_configuration!(state, base_joint, 0.0)
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

# Initialize state of water arm
state_water = MechanismState(mechanism_water)
# Make water state = arm state
impose_alpha_state_on_water(state_alpha, state_water)
# state_water_no_vel = deepcopy(state_water)

des_acc = similar(velocity(state_alpha))
des_acc[base_joint][1] = 0.0001
des_acc[shoulder_joint][1] = 0.
des_acc[elbow_joint][1] = 0.
des_acc[wrist_joint][1] = 0.

des_acc_water = similar(velocity(state_water))
impose_alpha_desacc_on_water(des_acc, des_acc_water)

0.0

In [89]:
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState)
    # Calculate inverse dynamics of alpha arm
    tau = inverse_dynamics(state_alpha, des_acc)
    # Find dynamic terms for water model
    impose_alpha_state_on_water(state_alpha, state_water)
    tau_water = inverse_dynamics(state_water, des_acc_water)

    # Set torques accordingly (controller)
    torques[velocity_range(state_alpha, base_joint)] .= tau[base_joint][1]
    torques[velocity_range(state_alpha, shoulder_joint)] .= tau[shoulder_joint][1]
    torques[velocity_range(state_alpha, elbow_joint)] .= tau[elbow_joint][1]
    torques[velocity_range(state_alpha, wrist_joint)] .= tau[wrist_joint][1]
    # torques[velocity_rangege(state_alpha, base_joint)] .= 0.
    # torques[velocity_range(state_alpha, shoulder_joint)] .= 0.
    # torques[velocity_range(state_alpha, elbow_joint)] .= 0.
    # torques[velocity_ran(state_alpha, wrist_joint)] .= 0.
end;

In [90]:
# ------------------------------------------------------------------------
#                      HYDRODYNAMICS CALCULATOR
# ------------------------------------------------------------------------
function hydro_calc!(hydro_wrenches::Dict{BodyID, Wrench{Float64}}, t, state_alpha::MechanismState)
    buoy_wrenches = []
    for i in 1:4
        bod = bodies(mechanism_alpha)[i+1]
        # println(bod)
        body_default_frame = default_frame(bod)
        def_to_cob = fixed_transform(bod, body_default_frame, cob_frames[i])
        lin_force_trans = transform(state_alpha, buoy_lin_forces[i], body_default_frame)
        # println(translation(def_to_cob))
        wrench = Wrench(Point3D(body_default_frame, translation(def_to_cob)), lin_force_trans)
        push!(buoy_wrenches, wrench)
        # buoy_wrench_dict[BodyID(bod)] = wrench
        hydro_wrenches[BodyID(bod)] = transform(state_alpha, wrench, root_frame(mechanism_alpha))
    end
end;

In [91]:
# ------------------------------------------------------------------------
#                          BUOYANCY CALCULATOR
# ------------------------------------------------------------------------
radius = 0.02
names = ["pt1", "pt2", "pt3", "pt4"]
hydro_wrenches = Dict{BodyID, Wrench{Float64}}()
buoy_wrenches = []
setelement!(mvis_alpha, default_frame(wrist_body))
for i in 1:4
    bod = bodies(mechanism_alpha)[i+1]
    # println(bod)
    body_default_frame = default_frame(bod)
    def_to_cob = fixed_transform(bod, body_default_frame, cob_frames[i])
    lin_force_trans = transform(state_alpha, buoy_lin_forces[i], body_default_frame)
    # println(translation(def_to_cob))
    wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_cob))), lin_force_trans)
    wrench_root = transform(state_alpha, wrench, root_frame(mechanism_alpha))
    println(wrench)
    setelement!(mvis_alpha, Point3D(body_default_frame, translation(inv(def_to_cob))), radius, names[i])
    push!(buoy_wrenches, wrench)
    # buoy_wrench_dict[BodyID(bod)] = wrench
    hydro_wrenches[BodyID(bod)] = wrench_root
end

# arrow_buoy_og = ArrowVisualizer(vis[:arrow1])
# setobject!(arrow_buoy_og)
# arrow_buoy_trans = ArrowVisualizer(vis[:arrow2])
# setobject!(arrow_buoy_trans)

# ----------------------------------------
# shoulder_ID = BodyID(shoulder_body)

# transform_from_base_to_wrist = relative_transform(state_alpha, root_frame(mechanism_alpha), default_frame(wrist_body))


Wrench expressed in "after_r5m_joint5":
angular: [-0.0005281507799999999, 0.00017605026, 0.0], linear: [0.0, 0.0, 0.17605026]
Wrench expressed in "after_r5m_joint4":
angular: [0.003826210918536263, -0.0010686597930247128, 0.13965669852657359], linear: [0.5310949728957587, 1.9131054592681316, 8.862804429034763e-5]
Wrench expressed in "after_r5m_joint3":
angular: [1.2634138699404597e-7, -0.0007333790253485766, 0.0029341478083292766], linear: [0.24451424975149616, -1.5459125785171403e-6, -1.0914927060799783e-5]
Wrench expressed in "after_r5m_joint2":
angular: [0.07123256591515872, -0.13037650447953966, 0.0], linear: [1.3303724946891802, 0.7268629175016196, -4.325788731817088e-5]
Dict{BodyID, Wrench{Float64}}(BodyID(5) => Wrench expressed in "world":
angular: [-1.085392815004707e-5, 0.10086493330361146, 5.8756476245367436e-18], linear: [7.905739638578161e-17, -3.627578731165549e-17, 1.5159883500000004], BodyID(4) => Wrench expressed in "world":
angular: [0.0007330951776923499, 0.0372966778

In [92]:
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 2
ts, qs, vs = simulate_with_ext_forces(state_alpha, final_time, hydro_calc!, simple_control!; Δt = 5e-4)
# ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

Simulation finished.


In [93]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
# open(mvis)
MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = .5)

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")


 done.
